In [1]:
import numpy as np
from PIL import Image
from PIL import ImageColor
import pandas as pd
from collections import Counter,OrderedDict
from itertools import dropwhile
from colormath.color_objects import sRGBColor, LabColor,LuvColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie2000,delta_e_cmc,delta_e_cie1976
import cv2 
import os 
import sys

In [2]:
color_to_class = {
    '#2d2c00':'stone',
    '#b87117':'granite',
    '#9fb816':'polished granite',
    '#e8e700':'diorite',
    '#ce2c15':'polished diorite',
    '#157200':'andesite',
    '#00b800':'polished andesite',
    '#5c2c44':'grass block',
    '#5aa115':'dirt',
    '#16a017':'coarse dirt',
    '#44e616':'podzol',
    '#5cd015':'cobblestone',
    '#5be700':'oak wood plank',
    '#452d15':'spruce wood plank',
    '#8b5c00':'birch wood plank',
    '#73a100':'jungle wood plank',
    '#2d165c':'acacia wood plank',
    '#a11400':'dark oak wood plank',
    '#16d000':'bedrock',
    '#a1d046':'sand',
    '#2d002b':'red sand',
    '#a12d44':'gravel',
    '#002d44':'gold ore',
    '#00b846':'iron ore',
    '#00a016':'coal ore',
    '#a1005d':'oak wood',
    '#d0015c':'spruce wood',
    '#45005b':'birch wood',
    '#73005b':'jungle wood',
    '#a2e645':'oak leaf',
    '#b8e645':'spruce leaf',
    '#00ce45':'birch leaf',
    '#15d043':'jungle leaf',
    '#5e732e':'glass',
    '#a1b846':'lapiz lazuli',
    '#e65b00':'sandstone',
    '#012c00':'chiseled sandstone',
    '#d0a000':'smooth sandstone',
    '#b7002c':'wool',
    '#894400':'brick',
    '#5e8a01':'bookshelf',
    '#73cf16':'mossy cobblestone',
    '#b7722d':'obsidian',
    '#725b17':'diamond ore',
    '#00742d':'furnace',
    '#892d16':'redstone ore',
    '#2d8a44':'ice block',
    '#005b46':'snow block',
    '#758a45':'pumpkin',
    '#d08917':'clay block',
    '#5b8a2c':'stone brick',
    '#165c44':'mossy stone brick',
    '#2d1644':'cracked stone brick',
    '#45d02d':'chiseled stone brick',
    '#cfa044':'iron bars',
    '#5dd02d':'vine',
    '#711500':'melon',
    '#740146':'mycelium',
    '#5c4444':'mycelium',
    '#ceb946':'acacia leaf',
    '#e7b844':'dark oak leaf',
    '#e8e644':'acacia wood',
    '#15005b':'dark oak wood',
    '#5dd001':'cactus',
    '#9f5169':'player',
    '#b9b715':'torch',
    '#2c2c46':'village road',
    '#8b2d45':'grass',
    '#b9005d':'oak wood',
    '#e7005c':'spruce wood',
    '#d08917':'clay block',
    '#001601':'podzol',
    '#2de700':'melon',
    '#004518':'wheat',
    '#2e1616':'wheat',
    '#170015':'wheat',
    '#592d00':'wheat',
    '#e57217':'wheat',
    '#437200':'wheat',
    '#e8165d':'wheat',
    '#171717':'wheat',
    '#16b900':'grass',
    '#e7162c':'fern',
    '#b94400':'dead bush',
    '#5d2d2d':'yellow flower',
    '#2d5b2c':'red flower',
    '#462c2b':'blue flower',
    '#2e2e2e':'purple flower',
    '#2e2e2c':'purple flower',
    '#732b2e':'white flower',
    '#725c2d':'red tulip',
    '#e6d000':'tall grass',
    '#d01715':'tall grass',
    '#445b2d':'orange tulip',
    '#885c2b':'white tulip',
    '#5a5c2b':'pink tulip',
    '#8a2c2c':'daisy',
    '#165d01':'tall fern',
    '#01a101':'tall fern',
    '#89e715':'rose bush',
    '#451700':'rose bush',
    '#b75c15':'peony',
    '#a2a017':'peony',
    '#8bb72e':'red mushroom',
    '#a2732d':'brown mushroom',
    '#7e54ba':'villager',
    '#ffffff':'spider',
    '#29c9bf':'horse',
    '#c26b3e':'sheep',
    '#afc324':'chicken',
    '#7051c9':'rabbit',
    '#00386b':'pig',
    '#263971':'wolf',
    '#9510cb':'zombie villager',
    '#fa75aa':'witch',
    '#e35fe6':'zombie',
    '#b65be7':'skeleton',
    '#5ba045':'bow',
    '#fd9cfd':'gold helmet',
    '#86d6ed':'creeper',
    '#c3bf15':'squid',
    '#2c5c44':'water',
    '#c24cac':'slime',
    '#d0a481':'mooshroom',
    '#2ce75a':'sword',
    '#b599e6':'ocelot',
    '#123650':'polar bear',
    '#000000':'lava',
    '#140000':'lava',
    '#e72c00':'lava',
    '#004600':'lava',
    '#b8b944':'lava',
    '#a20017':'cactus',
    '#2d4400':'water',
    '#450100':'water',
    '#164501':'water',
    '#2c0001':'water',
    '#facdae':'sky',
    '#608b18':'lama',
    '#23c33f':'mule',
    '#a05b16':'door',
    '#8ba216':'door',
    '#2e1500':'door',
    '#73e715':'door',
    '#5858d2':'donkey',
    '#002e2e':'fire',
    '#172c2d':'fire',
    '#b70043':'quartz',
    '#a04544':'quartz',
    '#5d0046':'glowstone',
    '#5b1643':'wool',
    '#164415':'wool',
    '#448a44':'nether brick',
    '#008945':'anvil',
    '#008a13':'wool',
    '#e7b715':'wool',
    '#2c0017':'wool',
    '#732d14':'rail',
    '#8be700':'rail',
    '#742d01':'wool',
    '#2cb714':'quartz',
    '#e7a046':'iron block',
    '#012d5c':'wool',
    '#8b002a':'stone slab',
    '#74462c':'stone slab',
    '#0abe1d':'sign',
    '#16002e':'chiseled quartz',
    '#00462e':'chiseled quartz',
    '#a11514':'door',
    '#5bb716':'torch',
    '#45b900':'wool',
    '#004443':'clay block',
    '#b8b62d':'prismarine',
    '#8a1617':'diamond block',
    '#8a155b':'sandstone',
    '#b7e600':'sandstone',
    '#17d02d':'sea lantern',
    '#2d0043':'glass',
    '#b8a12d':'glass',
    '#a1a22d':'glass',
    '#72a12d':'glass',
    '#460046':'glass',
    '#8aa12d':'glass',
    '#d0e62c':'glass',
    '#44a12e':'glass',
    '#150045':'glass',
    '#2ca12e':'glass',
    '#e6e62c':'glass',
    '#8a732f':'glass',
    '#17a22d':'glass',
    '#5ca12d':'glass',
    '#73732d':'glass',
    '#2ca146':'wool',
    '#5b7315':'rail',
    '#d11400':'rail',
    '#a2a000':'rail',
    '#73d044':'rail',
    '#897344':'clay block',
    '#157245':'clay block',
    '#ce5b46':'clay block',
    '#2d7245':'clay block',
    '#2c4444':'clay block',
    '#737244':'clay block',
    '#154545':'clay block',
    '#447244':'clay block',
    '#e72d45':'clay block',
    '#004443':'clay block',
    '#454545':'clay block',
    '#5b7244':'clay block',
    '#d02c44':'clay block',
    '#b82c46':'clay block',
    '#89b844':'lapiz lazuli block',
    '#e68b00':'door',
    '#005a00':'door',
    '#72d000':'crafting table',
    '#b7a100':'lamp',
    '#44b616':'nether brick',
    '#2dcf45':'nether wart block'
                 }


class_to_color = {v: ImageColor.getcolor(k, "RGB") for k, v in color_to_class.items()}


nums = np.arange(len(class_to_color))
rgb_to_class={ ImageColor.getcolor(k, "RGB"):v for k, v in color_to_class.items()}
normalize_colours = {ImageColor.getcolor(k, "RGB"):class_to_color[v]  for k, v in color_to_class.items()}
# print(class_to_color)
# print(normalize_colours)
print(rgb_to_class)

{(45, 44, 0): 'stone', (184, 113, 23): 'granite', (159, 184, 22): 'polished granite', (232, 231, 0): 'diorite', (206, 44, 21): 'polished diorite', (21, 114, 0): 'andesite', (0, 184, 0): 'polished andesite', (92, 44, 68): 'grass block', (90, 161, 21): 'dirt', (22, 160, 23): 'coarse dirt', (68, 230, 22): 'podzol', (92, 208, 21): 'cobblestone', (91, 231, 0): 'oak wood plank', (69, 45, 21): 'spruce wood plank', (139, 92, 0): 'birch wood plank', (115, 161, 0): 'jungle wood plank', (45, 22, 92): 'acacia wood plank', (161, 20, 0): 'dark oak wood plank', (22, 208, 0): 'bedrock', (161, 208, 70): 'sand', (45, 0, 43): 'red sand', (161, 45, 68): 'gravel', (0, 45, 68): 'gold ore', (0, 184, 70): 'iron ore', (0, 160, 22): 'coal ore', (161, 0, 93): 'oak wood', (208, 1, 92): 'spruce wood', (69, 0, 91): 'birch wood', (115, 0, 91): 'jungle wood', (162, 230, 69): 'oak leaf', (184, 230, 69): 'spruce leaf', (0, 206, 69): 'birch leaf', (21, 208, 67): 'jungle leaf', (94, 115, 46): 'glass', (161, 184, 70): 'la

In [33]:
rev_multidict = {}
for key, value in color_to_class.items():
    rev_multidict.setdefault(value, set()).add(key)
temp=[key for key, values in rev_multidict.items() if len(values) > 1]
print(temp)

['grass block', 'podzol', 'oak wood', 'spruce wood', 'glass pane', 'stone slab', 'clay block', 'melon', 'cactus', 'acacia wood', 'dark oak wood', 'tall grass', 'water', 'lava', 'grass road', 'wheat']


In [3]:
print(color_to_class[(83, 205, 0)])

oak wood planks


In [10]:

file1='colourmap_video.mp4'
file2='video.mp4'
file_directory = r'C:\Users\azuzo\Desktop\Malmo\videos\d658626a-0620-40b3-afab-b8d09e165381'
file1= os.path.join(file_directory,file1)
file2= os.path.join(file_directory,file2)
print(file1,file2)
def seperate_frames(file,frame_dir):
    cam = cv2.VideoCapture(file)
    
    frame_path=os.path.join(frame_dir,'frames')
    colourmap_path=os.path.join(frame_path,'colourmap_video')
    video_path=os.path.join(frame_path,'video')

    try:

        # creating a folder named data
        if not os.path.exists(frame_dir):
            os.makedirs(frame_dir)
        if not os.path.exists(frame_path):
            os.makedirs(frame_path)
        if not os.path.exists(video_path):
            os.makedirs(video_path)
        if not os.path.exists(colourmap_path):
            os.makedirs(colourmap_path)
    
    # if not created then raise error
    except OSError:
        print ('Error: Creating directory of data')

    # frame
    currentframe = 0

    while(True):

        # reading from frame
        ret,frame = cam.read()

        if ret:
            # if video is still left continue creating images
            if 'colourmap' in file:
                name = colourmap_path + '/' + str(currentframe) + '.png'
                print ('Creating...' + name)

                # writing the extracted images
                cv2.imwrite(name, frame)

                # increasing counter so that it will
                # show how many frames are created
                currentframe += 1
            else:
                name = video_path + '/' + str(currentframe) + '.png'
                print ('Creating...' + name)

                # writing the extracted images
                cv2.imwrite(name, frame)

                # increasing counter so that it will
                # show how many frames
                currentframe += 1
                
        else:
            break

    # Release all space and windows once done
    cam.release()
    cv2.destroyAllWindows()
# seperate_frames(file1,file_directory)
# seperate_frames(file2,file_directory)

C:\Users\azuzo\Desktop\Malmo\videos\d658626a-0620-40b3-afab-b8d09e165381\colourmap_video.mp4 C:\Users\azuzo\Desktop\Malmo\videos\d658626a-0620-40b3-afab-b8d09e165381\video.mp4


In [540]:

def rgb2hex(r,g,b):
    return '#%02x%02x%02x' % (r, g, b)

def hex2rgb(hexcode):
    return tuple(map(ord,hexcode[1:].decode('hex')))



def most_common_rgb(directory):
    directory = os.path.join(directory,'frames\colourmap_video')
    
    counter=0
    images=os.listdir(directory)
    temp=[]
    for image in images:
        image=image[:-4]
        temp.append(image)
    temp = [int(x) for x in temp]
    temp=sorted(temp)
    temp = [str(x) for x in temp]
    temp = [x+'.jpg' for x in temp]
    images=temp
#     print(images)
    for image in images:
        pixels=[]
        counter+=1
        image_path= os.path.join(directory,image)
        img = Image.open(image_path)
#         img=img.convert('RGB')
        print(img.mode)
        width, height = img.size
        for y in range(height):
            for x in range(width):
                r,g,b=img.getpixel((x,y))
              
                color = rgb2hex(r,g,b)
                pixels.append(color)
        print(image_path)
        unique,pos = np.unique(pixels,return_inverse=True)
        counts = np.bincount(pos)
        if len(counts)>=6:
            maxpos = np.argpartition(counts, -6)[-6:]
            print(unique[maxpos],counts[maxpos])
        else:
            maxpos=counts.argmax()
            print(unique[maxpos],counts[maxpos])
            
#         colors, frequency=np.unique(pixels, return_counts=True)
#         print(colors[:])
#         print(colors[-1])
#         print()
    
# most_common_rgb(r'C:\Users\azuzo\Desktop\Malmo\videos\255f07a1-505c-497b-9a47-e1e09d7a0ef6')

RGB
C:\Users\azuzo\Desktop\Malmo\videos\255f07a1-505c-497b-9a47-e1e09d7a0ef6\frames\colourmap_video\0.jpg
['#5b1740' '#fbceb1' '#5d1545' '#facdae' '#e7b715' '#5b1643'] [  1295   2816   4265 267520  64512 423078]
RGB
C:\Users\azuzo\Desktop\Malmo\videos\255f07a1-505c-497b-9a47-e1e09d7a0ef6\frames\colourmap_video\1.jpg
['#007344' '#008a13' '#5cd015' '#2d2c00' '#facdae' '#e65b00'] [  7552  23513 167720  19779 345197  18044]
RGB
C:\Users\azuzo\Desktop\Malmo\videos\255f07a1-505c-497b-9a47-e1e09d7a0ef6\frames\colourmap_video\2.jpg
['#007344' '#2d2c00' '#008a13' '#e65b00' '#facdae' '#5cd015'] [  7552  22067  25677 124442 126043 169648]
RGB
C:\Users\azuzo\Desktop\Malmo\videos\255f07a1-505c-497b-9a47-e1e09d7a0ef6\frames\colourmap_video\3.jpg
['#007344' '#2d2c00' '#008a13' '#5cd015' '#e65b00' '#facdae'] [  7552  22067  25677 169648 135059 105895]
RGB
C:\Users\azuzo\Desktop\Malmo\videos\255f07a1-505c-497b-9a47-e1e09d7a0ef6\frames\colourmap_video\4.jpg
['#007344' '#facdae' '#5cd015' '#e65b00' '#2d2

RGB
C:\Users\azuzo\Desktop\Malmo\videos\255f07a1-505c-497b-9a47-e1e09d7a0ef6\frames\colourmap_video\45.jpg
['#444645' '#454543' '#facdae' '#5b7244' '#004443' '#454545'] [   990   1025   1224  79523  84606 571253]
RGB
C:\Users\azuzo\Desktop\Malmo\videos\255f07a1-505c-497b-9a47-e1e09d7a0ef6\frames\colourmap_video\46.jpg
['#b82c46' '#facdae' '#e65b00' '#d02c44' '#5b7244' '#454545'] [ 17366  23430  86180 159534 162948 172284]
RGB
C:\Users\azuzo\Desktop\Malmo\videos\255f07a1-505c-497b-9a47-e1e09d7a0ef6\frames\colourmap_video\47.jpg
#5b7244 786420
RGB
C:\Users\azuzo\Desktop\Malmo\videos\255f07a1-505c-497b-9a47-e1e09d7a0ef6\frames\colourmap_video\48.jpg
['#facdae' '#b82c46' '#5b7244' '#e65b00' '#d02c44' '#454545'] [ 24462  78373 172677  80965 173854 120046]
RGB
C:\Users\azuzo\Desktop\Malmo\videos\255f07a1-505c-497b-9a47-e1e09d7a0ef6\frames\colourmap_video\49.jpg
['#d32b44' '#cf2d44' '#d22c44' '#b82c46' '#5b7244' '#d02c44'] [   526    854    875   5468 103945 654130]
RGB
C:\Users\azuzo\Desktop

In [6]:
from scipy.spatial.distance import cdist

colours = []
color_rgb_to_class=dict()

for color in list(color_to_class.keys()):
    rgb_color =ImageColor.getrgb(color)
    color_rgb_to_class[rgb_color]=color_to_class[color]
    colours.append(rgb_color)
colours = np.asarray(colours)
rgb_classes= np.array([(0,0,0)]+list(set(normalize_colours.values())))
for rgb in rgb_classes:
    if rgb.sum()==0:
        print (rgb)
total_classes=len(rgb_classes)

def create_masks(directory):
    mask_dir=os.path.join(directory,'frames\mask')
    directory = os.path.join(directory,'frames\colourmap_video')
    images=os.listdir(directory)
    temp=[]
    for image in images:
        image=image[:-4]
        temp.append(image)
    temp = [int(x) for x in temp]
    temp=sorted(temp)
    temp = [str(x) for x in temp]
    temp = [x+'.png' for x in temp]
    images=temp
    for image in images:
        image_path= os.path.join(directory,image)
        img = np.asarray(Image.open(image_path))
        
        im = colours[cdist(img.reshape(-1,3),colours).argmin(1)].reshape(img.shape)
        height,width,_=img.shape
        print(len(set(normalize_colours.values())))
        for i in range(height):
            for j in range(width):
                r,g,b=normalize_colours[tuple(im[i,j])]
                im[i,j]=np.array([r,g,b])
        im = Image.fromarray(im.astype('uint8'),'RGB')
#         print(Counter(im.getdata()))
#         im.show()
        print(image[:-4])
        if not os.path.exists(mask_dir):
            os.makedirs(mask_dir)
        outdir= os.path.join(mask_dir,image[:-4]+'.png')
        print(outdir)
        height,width,_=img.shape
        mask=np.zeros((height,width),dtype=np.uint8)
        for i in range(total_classes):
            indicies= np.where(np.all(rgb_classes[i]==im,axis=-1))
#             print(indicies)
            mask[indicies[0],indicies[1]]=i
        rgbMask= rgb_classes[mask].astype('uint8')
        
        print('rgb Mask',rgbMask.shape)
        rgbMask=Image.fromarray(rgbMask)
#         rgbMask.show()
        mask=Image.fromarray(mask)
#         mask.show()
#         print(mask.shape)
#         print(rgb_to_class[tuple(rgb_classes[11])])
        mask.save(outdir)
        
# create_masks(r'C:\Users\azuzo\Desktop\Malmo\videos\d658626a-0620-40b3-afab-b8d09e165381')

[0 0 0]


In [9]:
def greyscale_to_rgb_mask(image):
    rgbMask= rgb_classes[image].astype('uint8')
    rgbMask=Image.fromarray(rgbMask)
    rgbMask.save(r'C:\Users\azuzo\Desktop\Malmo\videos\d658626a-0620-40b3-afab-b8d09e165381\frames\colormap_2186.png')
    rgbMask.show()
path=r'C:\Users\azuzo\Desktop\Malmo\videos\d658626a-0620-40b3-afab-b8d09e165381\frames\mask\2186.png'
img = Image.open(path)
greyscale_to_rgb_mask(img)

In [67]:
from PIL import Image
import imagehash
def remove_dups(directory):
    
    mask_dir=os.path.join(directory,'frames\mask')
    directory = os.path.join(directory,'frames\colourmap_video')
    images=os.listdir(directory)
    # image_fns : List of training image files
    img_hashes = {}
    epsilon = 10
    
    for image_fn1, image_fn2 in zip(images, images[::-1]):
        image_fn1=os.path.join(directory,image_fn1)
        image_fn2= os.path.join(directory,image_fn2)
        if image_fn1 == image_fn2:
            continue

        hash1 = imagehash.average_hash(Image.open(image_fn1))
        hash2 = imagehash.average_hash(Image.open(image_fn2))
        if hash1 - hash2 < epsilon:
            print( '{} is near duplicate of {}'.format(image_fn1.split('\\')[-1], image_fn2.split('\\')[-1]) )
# remove_dups(r'C:\Users\azuzo\Desktop\Malmo\videos\d658626a-0620-40b3-afab-b8d09e165381')

10000.png is near duplicate of 9994.png
10565.png is near duplicate of 943.png
10575.png is near duplicate of 942.png
10585.png is near duplicate of 941.png
10595.png is near duplicate of 940.png
10605.png is near duplicate of 939.png
117.png is near duplicate of 8296.png
11786.png is near duplicate of 8209.png
120.png is near duplicate of 7996.png
12266.png is near duplicate of 7729.png
1658.png is near duplicate of 6974.png
1865.png is near duplicate of 6767.png
197.png is near duplicate of 6662.png
2313.png is near duplicate of 6317.png
2314.png is near duplicate of 6316.png
238.png is near duplicate of 6251.png
2598.png is near duplicate of 6033.png
2599.png is near duplicate of 6032.png
2751.png is near duplicate of 5880.png
2753.png is near duplicate of 5879.png
2754.png is near duplicate of 5878.png
2755.png is near duplicate of 5877.png
2768.png is near duplicate of 5864.png
2769.png is near duplicate of 5863.png
2770.png is near duplicate of 5861.png
2778.png is near duplicate